In [ ]:
!pip install opencv-python matplotlib numpy scipy

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2, fftshift, ifftshift
import os

input_dir = 'exercise'
output_dir = 'output_fft'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def fast_fourier(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f = fft2(gray)
    fshift = fftshift(f)
    magnitude = 20 * np.log(np.abs(fshift) + 1)
    return np.uint8(cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX))

def butterworth_filter(img, d0, n=2, highpass=False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rows, cols = gray.shape
    u = np.arange(rows)
    v = np.arange(cols)
    u[u > rows // 2] -= rows
    v[v > cols // 2] -= cols
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2 + V**2)

    if highpass:
        H = 1 / (1 + (d0 / (D + 1e-5))**(2 * n))
    else:
        H = 1 / (1 + (D / d0)**(2 * n))

    F = fftshift(fft2(gray))
    G = H * F
    g = np.abs(ifft2(ifftshift(G)))
    return np.uint8(cv2.normalize(g, None, 0, 255, cv2.NORM_MINMAX))

def butterworth_lowpass(img):
    return butterworth_filter(img, d0=50, n=2, highpass=False)

def butterworth_highpass(img):
    return butterworth_filter(img, d0=50, n=2, highpass=True)

In [ ]:
transformations = {
    'F': ('Fast Fourier Transform', fast_fourier),
    'L': ('Butterworth Lowpass Filter', butterworth_lowpass),
    'H': ('Butterworth Highpass Filter', butterworth_highpass)
}

print("=== Frequency Domain Transformations ===")
for key, (name, _) in transformations.items():
    print(f"{key} - {name}")

choice = input("Nhập lựa chọn của bạn (F/L/H): ").upper()

if choice not in transformations:
    print("Lựa chọn không hợp lệ.")
else:
    name, transform_fn = transformations[choice]
    print(f"Đang áp dụng: {name}")

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            path = os.path.join(input_dir, filename)
            img = cv2.imread(path)

            result = transform_fn(img)
            out_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_{choice}.png")
            cv2.imwrite(out_path, result)

            plt.figure(figsize=(10, 4))
            plt.subplot(1, 2, 1)
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title("Original")
            plt.axis("off")

            plt.subplot(1, 2, 2)
            plt.imshow(result, cmap='gray')
            plt.title(name)
            plt.axis("off")
            plt.show()

    print(f"✅ Ảnh đã lưu tại: '{output_dir}'")